# Chapter8-1

## 文埋め込みモデルの実装

教師なしSimCSE, 教師ありSimCSEを用いてモデルの学習、評価を行う

今回はJSNLIのデータセットを使用して学習を行う

モデルの評価はJSTSを使用してモデルの評価を行う

## 教師なしSimCSE

### ライブラリのインストール

In [1]:
!pip install datasets scipy transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 68.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 

In [2]:
# 乱数シードを固定
from transformers.trainer_utils import set_seed

set_seed(42)

### データセットの読み込み、前処理

In [3]:
from datasets import load_dataset

# データセットの取得
unsup_train_dataset = load_dataset(
    "llm-book/jawiki-sentences", split="train"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for llm-book/jawiki-sentences contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/jawiki-sentences.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/24387500 [00:00<?, ? examples/s]

In [4]:
# 学習セットの形式、事例数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [5]:
# 50件の訓練内容を表示
for i, text in enumerate(unsup_train_dataset[:50]["text"]):
  print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。
5 "&" はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。
6 結果として、アルファベットの復唱の最後は "X, Y, Z, and per se and" という形になった。
7 この最後のフレーズが繰り返されるうちに "ampersand" と訛っていき、この言葉は1837年までには英語の一般的な語法となった。
8 アンドレ=マリ・アンペールがこの記号を自身の著作で使い、これが広く読まれたため、この記号が "Ampère's and" と呼ばれるようになったという誤った語源俗説がある。
9 アンパサンドの起源は1世紀の古ローマ筆記体にまで遡ることができる。
10 古ローマ筆記体では、E と T はしばしば合字として繋げて書かれていた(左図「アンパサンドの変遷」の字形1)。それに続く、流麗さを増した新ローマ筆記体では、様々な合字が極めて頻繁に使われるようになった。
11 字形2と3は4世紀中頃における et の合字の例である。
12 その後、9世紀のカロリング小文字体に至るラテン文字の変遷の過程で、合字の使用は一般には廃れていった。
13 しかし、et の合字は使われ続け、次第に元の文字がわかりにくい字形に変化していった(字形4から6)。
14 現代のイタリック体のアンパサンドは、ルネサンス期に発展した筆記体での et の合字に遡る。
15 1455年のヨーロッパにおける印刷技術の発明以降、印刷業者はイタリック体とローマ筆記体のアンパサンドの両方を多用するようになった。
16

43行目のように空白になっている部分は空行

今回の学習では不要なため処理除く必要がある

In [6]:
# 空行の事例を削除
unsup_train_dataset = unsup_train_dataset.filter(
    lambda example: example["text"].strip() != ""
)

Filter:   0%|          | 0/24387500 [00:00<?, ? examples/s]

In [7]:
# 43行目の空行が削除されているか確認
print(unsup_train_dataset[43]["text"] != "")

True


In [8]:
# 学習セットをシャッフルし、100万事例を抽出
unsup_train_dataset = unsup_train_dataset.shuffle().select(
    range(1000000)
)

# シャッフルされた学習セットをディスクに書き込む (パフォーマンスの低下防止)
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [9]:
# 抽出された学習セットの数を確認
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000000
})


In [10]:
# 文の順序がシャッフルされているか確認
for i, text in enumerate(unsup_train_dataset[:10]["text"]):
  print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。
5 さらにマップ上で最大8つまでしか建築できず(司令官アビリティの”解体”か設置したプレイヤー自らが出向いて解体する必要がある)
6 特に誉淳が1827年から作成した『古瓦譜』は畿内で600点以上の拓本を蒐集し、瓦当文様に着目したうえで編年を試みている。
7 マルクス主義者を広言し、メキシコ共産党の敵であり味方であった。
8 ICHILLIN'(アイチリン、朝: 아이칠린)は、韓国の7人組女性アイドルグループ。
9 マークVIは1983年にモデルサイクルを終了し、1984年のマークVII(英語版)はフルサイズセグメントから撤退し、マークシリーズは異なるセグメントに移行した。


### 検証用、テスト用のデータセットを読み込み

In [11]:
DATASET = "llm-book/JGLUE"

# 検証セットのダウンロード
valid_dataset = load_dataset(
    DATASET, name="JSTS", split="train"
)

# テストセットのダウンロード
test_dataset = load_dataset(
    DATASET, name="JSTS", split="train"
)



The repository for llm-book/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/JGLUE.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

### トークナイザ, collate関数の準備

In [12]:
from transformers import AutoTokenizer

base_model_name = "cl-tohoku/bert-base-japanese-v3"

# トークナイザを初期化
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [13]:
import torch
from torch import Tensor
from transformers import BatchEncoding

def unsup_train_collate_fn(
    examples: list[dict],
)-> dict[str, BatchEncoding | Tensor]:
  """ 教師なしSimCSEの学習セットのミニバッチを作成 """

  # ミニバッチに含まれる文にトークナイザを適用
  tokenized_texts = tokenizer(
      [example["text"] for example in examples],
      padding=True,
      truncation=True,
      max_length=32,
      return_tensors="pt"
  )

  # 文と文の類似度行列における正例ペアの位置を表すTensorを作成
  # 行列のi行目の事例に対してi列目の事例との組が正例ペアとなる
  labels = torch.arange(len(examples))

  return {
      "tokenized_texts_1": tokenized_texts,
      "tokenized_texts_2": tokenized_texts,
      "labels": labels
  }


In [14]:
def eval_collate_fn(
    examples: list[dict],
) -> dict[str, BatchEncoding | Tensor]:
  """ SimCSEの検証・テストセットのミニバッチを作成 """

  # ミニバッチの文ペアに含まれる文それぞれトークナイザを適用
  tokenized_texts_1 = tokenizer(
      [example["sentence1"] for example in examples],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )

  tokenized_texts_2 = tokenizer(
      [example["sentence2"] for example in examples],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt"
  )

  labels = torch.arange(len(examples))

  # 類似度スコアのTensorを作成
  label_scores = torch.tensor(
      [example["label"] for example in examples]
  )

  return {
      "tokenized_texts_1": tokenized_texts_1,
      "tokenized_texts_2": tokenized_texts_2,
      "labels": labels,
      "label_scores": label_scores
  }

1つの事例に2つ以上の文が含まれているときは、トークナイザをデータの読み込み時に適用するとあとの処理が複雑になる

### モデルの準備

SimCSEはtransformersには含まれていない

$\rightarrow$ `torch.nn.Module`を定義して実装

エンコーダの上にMLP層をヘッドとして加えるだけ

In [15]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSE(nn.Module):
  """ SimCSEのモデル """

  def __init__(
      self,
      base_model_name: str,
      mlp_only_train: bool = False,
      temperature: float = 0.05,
  ):
    """ モデルの初期化 """

    super().__init__()

    # モデル名からエンコーダを初期化
    self.encoder = AutoModel.from_pretrained(base_model_name)

    # MLP層の次元数
    self.hidden_size = self.encoder.config.hidden_size

    # MLP層の線形層
    self.dense = nn.Linear(self.hidden_size, self.hidden_size)

    # MLP層の活性化関数
    self.activation = nn.Tanh()

    # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
    self.mlp_only_train = mlp_only_train

    # 交差エントロピー損失の計算時に使用する温度
    self.temperature = temperature

  def encode_texts(self, tokenized_texts: BatchEncoding) -> Tensor:
    """ エンコーダを用いて文をベクトルに変換 """

    # トークナイズされた分をエンコーダに入力
    encoded_texts = self.encoder(**tokenized_texts)

    # モデルの最終層の出力の[CLS]トークンのベクトルを取り出す
    encoded_texts = encoded_texts.last_hidden_state[:, 0]

    # フラグがTrueかつ学習時の時, MLP層の変換処理を適用せずにベクトルを返す
    if self.mlp_only_train and not self.training:
      return encoded_texts

    # MLP層の変換処理
    encoded_texts = self.dense(encoded_texts)
    encoded_texts = self.activation(encoded_texts)

    return encoded_texts

  def forward(
      self,
      tokenized_texts_1: BatchEncoding,
      tokenized_texts_2: BatchEncoding,
      labels: Tensor,
      label_scores: Tensor | None = None,
  ) -> ModelOutput:
    """ モデルの前向き計算を実施 """

    # 文のベクトルに変換
    encoded_texts_1 = self.encode_texts(tokenized_texts_1)
    encoded_texts_2 = self.encode_texts(tokenized_texts_2)

    # 文の類似度行列の計算
    sim_matrix = F.cosine_similarity(
        encoded_texts_1.unsqueeze(1),
        encoded_texts_2.unsqueeze(2),
        dim=2,
    )

    # 交差エントロピー損失を求める
    loss = F.cross_entropy(sim_matrix / self.temperature, labels)

    # 正例ペアに対するスコアを類似度行列から取り出す
    positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
    positive_scores = torch.masked_select(sim_matrix, positive_mask)

    # 負例ペアに対するスコアを類似度行列から取り出す
    negative_mask = positive_mask
    negative_scores = torch.masked_select(sim_matrix, negative_mask)

    return ModelOutput(loss=loss, scores=positive_scores)

# SimCSEモデルを初期化
unsup_model = SimCSE(base_model_name, mlp_only_train=True)


config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

### Trainerの準備

In [16]:
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
  """
  モデルが予測したスコアと評価用データスコアの
  スピアマンの順位相関係数を計算
  """

  scores = p.predictions
  labels, label_scores = p.label_ids

  spearman = spearmanr(scores, label_scores).statistic

  return {"spearman": spearman}

In [17]:
from transformers import TrainingArguments

# SimCSEのハイパーパラメータを設定
unsup_training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=3e-5,
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=250,                 # 250ステップ学習を行うごとに検証セットを使用して評価
    logging_steps=250,              # 250ステップ学習を行うごとにログを出力
    save_steps=250,                 # 250ステップ学習を行うごとにモデルを保存
    save_total_limit=1,             # 保存するチェックポイントの最大数
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="spearman",
    remove_unused_columns=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# Trainerを初期化
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import Trainer

class SimCSETrainer(Trainer):
  """ SimCSEのTrainer """

  def get_eval_dataloader(
      self, eval_dataset: Dataset | None = None
  )-> DataLoader:
    """
    検証・テストセットのDataLoaderでeval_collate_fnを使うように
    Trainerのget_eval_dataloaderをオーバーライド
    """

    if eval_dataset is None:
      eval_dataset = self.eval_dataset

    return DataLoader(
        eval_dataset,
        batch_size=64,
        collate_fn=eval_collate_fn,
        pin_memory=True,
    )

# 教師なしSimCSEのTrainerを初期化
unsup_trainer = SimCSETrainer(
    model=unsup_model,
    args=unsup_training_args,
    data_collator=unsup_train_collate_fn,
    train_dataset=unsup_train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)


In [19]:
#学習の実行
unsup_trainer.train()

Step,Training Loss,Validation Loss,Spearman
250,6.676900,7.177592,-0.012815
500,6.663900,7.121831,0.004819
750,6.654700,7.018140,-0.005539
1000,6.646800,6.991966,-0.001182
1250,6.644000,6.926537,0.005801
1500,6.646000,6.996048,0.010313
1750,6.643900,6.863707,-0.008785
2000,6.642300,6.842277,-0.014603
2250,6.641600,7.018536,0.009176
2500,6.640600,6.979576,0.011471


TrainOutput(global_step=15625, training_loss=6.6402240703125, metrics={'train_runtime': 5405.538, 'train_samples_per_second': 184.995, 'train_steps_per_second': 2.891, 'total_flos': 0.0, 'train_loss': 6.6402240703125, 'epoch': 1.0})

### 性能評価

学習を行った人は検証セットでのスコアが一番高かったモデルが読み込まれている

ここでは検証セット、テストセットを使って最終評価をする

In [20]:
# 検証セットを使って評価
unsup_trainer.evaluate(valid_dataset)

{'eval_loss': 6.979842662811279,
 'eval_spearman': 0.016049384313863085,
 'eval_runtime': 14.6874,
 'eval_samples_per_second': 847.733,
 'eval_steps_per_second': 13.277,
 'epoch': 1.0}

In [21]:
# テストセットを使って評価
unsup_trainer.evaluate(test_dataset)

{'eval_loss': 6.979842662811279,
 'eval_spearman': 0.016049384313863085,
 'eval_runtime': 14.9693,
 'eval_samples_per_second': 831.767,
 'eval_steps_per_second': 13.027,
 'epoch': 1.0}

### トークナイザ、モデルの保存


In [22]:
# SimCSEのエンコーダを保存
encoder_path = "outputs/encoder"
unsup_model.encoder.save_pretrained(encoder_path)
tokenizer.save_pretrained(encoder_path)

('outputs/encoder/tokenizer_config.json',
 'outputs/encoder/special_tokens_map.json',
 'outputs/encoder/vocab.txt',
 'outputs/encoder/added_tokens.json')

In [24]:
# Googleドライブに保存(任意)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
#保存されたモデルをGoogleドライブのフォルダにコピー
!mkdir -p drive/MyDrive/Learning-LLM/
!cp -r outputs/encoder drive/MyDrive/Learning-LLM/